In [ ]:
import sys 
import os
import cobra
import cplex 
import libsbml
import copy
from pathlib import Path
import memote
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
import path 
import pandas as pd
import numpy as np
from scipy.stats import zscore
from itertools import combinations
from scipy.stats import kruskal, mannwhitneyu
import multiprocessing
from itertools import combinations, product
from multiprocessing import Pool
from sklearn.decomposition import PCA
import mplcursors
import math
import re
import itertools
from scipy.stats import linregress


from sklearn.linear_model  import LinearRegression

#Change working dir first, ty ChatGPT, much loves
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src/")

import model_initialize as mi
import model_manipulation as mm


#Set solver to gurobi
config = cobra.Configuration()
config.solver = 'gurobi'

#Read 2-cell model
wt_model = cobra.io.read_sbml_model("./model/ios2164_2cell.xml")
trans_model = cobra.io.read_sbml_model("./model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6


#Add trans reactions to trans_model
mi.add_trans_reactions(trans_model)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-09


Pipeline breakdown:

1. Load CSVs and integrate to single dataframe
2. Generate correlation matrices for each dataframe
3. Obtain list of reactions of interest first 
    Glutamate and MDHys_M
    RBPCs_BS
    MDHys_BS
4. Initialize subplots for each reaction of interest
5. Plot each scatter plot corresponding to the reactions of interest
6. Plot also correlation matrix for reactions in the M stromal compartment only



In [ ]:

def merge_dataframes(dataframes, names):
    merged_df = pd.DataFrame()
    
    for i, df in enumerate(dataframes):
        # Get the corresponding name from the names list
        df_name = names[i]
        
        # Add a new column with the dataframe name
        df['sample'] = df_name
        
        # Merge dataframes
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    # Remove NAs
    merged_df = merged_df.fillna(1e-7)
    
    return merged_df

def visualize_correlation_matrix(df, sort_by=None, ascending=False):
    """
    Calculates and visualizes the correlation matrix based on flux measurements in a dataframe.

    Parameters:
        - df (pandas.DataFrame): The input dataframe containing flux measurements.
        - sort_by (str or None): Column name to sort the correlation matrix by. Default is None.
        - ascending (bool): Whether to sort in ascending order. Default is False.
    """

    # Calculate the correlation matrix
    correlation_matrix = df.corr()

    # Sort the correlation matrix if sort_by is specified
    if sort_by is not None:
        correlation_matrix = correlation_matrix.sort_values(by=sort_by, ascending=ascending)

    # Create a heatmap of the correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', square=True)

    # Set plot title
    plt.title('Correlation Matrix')

    # Display the heatmap
    plt.show()

def filter_correlation_pairs(correlation_matrix, threshold, filter_elements):
    """
    Filters out specific pairs in the correlation matrix based on a specified absolute value threshold,
    removes pairs that share columns, removes symmetrical results, and returns the rows where the filter elements are found in either of the two variable columns.

    Parameters:
        - correlation_matrix (pandas.DataFrame): The correlation matrix.
        - threshold (float): The absolute value threshold to filter the correlation coefficients.
        - filter_elements (list): The elements to filter out from the correlation matrix.

    Returns:
        - filtered_pairs (pandas.DataFrame): The table showing the filtered pairs and their correlation coefficients,
                                             where the filter elements are found in either of the two variable columns,
                                             with symmetrical results removed.
    """
    # Filter out pairs based on the threshold
    filtered_pairs = correlation_matrix.unstack().reset_index()
    filtered_pairs = filtered_pairs.rename(columns={'level_0': 'Variable A', 'level_1': 'Variable B', 0: 'Correlation'})
    filtered_pairs = filtered_pairs[filtered_pairs['Correlation'].abs() >= threshold]

    # Remove pairs that share columns
    filtered_pairs = filtered_pairs[~(filtered_pairs['Variable A'] == filtered_pairs['Variable B'])]

    # Remove symmetrical results
    filtered_pairs = filtered_pairs[filtered_pairs['Variable A'] < filtered_pairs['Variable B']]

    # Filter rows where the filter elements are found in either of the two variable columns
    filtered_pairs = filtered_pairs[(filtered_pairs['Variable A'].isin(filter_elements)) | (filtered_pairs['Variable B'].isin(filter_elements))]
    
    # Sort the filtered pairs by correlation coefficient in descending order
    filtered_pairs = filtered_pairs.sort_values('Correlation', ascending=False)
    
    filtered_pairs = filtered_pairs.reset_index()
    

    return filtered_pairs

def further_filter_correlation_pairs(filtered_pairs, additional_filter_elements):
    """
    Further filters the previously filtered correlation pairs based on an additional list of filter elements.

    Parameters:
        - filtered_pairs (pandas.DataFrame): The DataFrame containing the filtered pairs from the initial filtering.
        - additional_filter_elements (list): The additional elements to filter the pairs based on.

    Returns:
        - further_filtered_pairs (pandas.DataFrame): The table showing the further filtered pairs and their correlation coefficients,
                                                    where the additional filter elements are found in either of the two variable columns.
    """
    # Filter rows where the additional filter elements are found in either of the two variable columns
    further_filtered_pairs = filtered_pairs[(filtered_pairs['Variable A'].isin(additional_filter_elements)) | (filtered_pairs['Variable B'].isin(additional_filter_elements))]
    
    return further_filtered_pairs



def plot_scatter_regression(df, column_x, column_y):
    """
    Generates a scatterplot and a linear regression line between two columns in a dataframe.

    Parameters:
        - df (pandas.DataFrame): The input dataframe.
        - column_x (str): The column name for the X-axis.
        - column_y (str): The column name for the Y-axis.
    """

    # Extract X and Y values from the dataframe
    X = df[column_x].values.reshape(-1, 1)
    Y = df[column_y].values

    # Fit linear regression model
    regression = LinearRegression()
    regression.fit(X, Y)

    # Predict Y values based on the regression line
    Y_pred = regression.predict(X)

    # Plot the scatterplot and regression line
    plt.scatter(X, Y, color='blue', label='Actual')
    plt.plot(X, Y_pred, color='red', label='Regression Line')
    plt.xlabel(column_x)
    plt.ylabel(column_y)
    plt.title('Scatterplot with Linear Regression Line')
    plt.legend()
    plt.show()
    
    
    

def get_aggregate_reactions_mets(mets_to_search='', model=trans_model):
    '''
    This function looks for reactions in the model that contain specific metabolites.
    '''
    if type(mets_to_search) != list:
        print('mets not in list form')
        return None
    # This codeblock gets all the metabolites in all compartments
    met_list = list()
    for mets in model.metabolites:
        for i in mets_to_search:
            if i in mets.id:
                if mets.id.startswith(i):
                    met_list.append(mets)

    # This codeblock gets all the reactions that have the metabolite in their reaction
    rxn_list = list()
    for mets in met_list:
        for rxns in model.reactions:
            if mets in rxns.metabolites:
                rxn_list.append(rxns.id)
                
    # Output a dictionary with 'met_list' as keys and 'rxn_list' as values
    met_rxn_dict = {met.id: [rxn for rxn in rxn_list if met in model.reactions.get_by_id(rxn).metabolites]
                    for met in met_list}

    return met_rxn_dict
            



In [3]:
# #Test the script on one CSV

# wt_250 = load_csv_and_cleanup('./flux_results/flux_sampling/flux_sample_WT_250_Relaxed_loopless_FVA_100kT.csv' )
# wt_750 = load_csv_and_cleanup('./flux_results/flux_sampling/flux_sample_WT_750_Relaxed_loopless_FVA_100kT_reran.csv')
# wt_1500 = load_csv_and_cleanup('./flux_results/flux_sampling/flux_sample_WT_1500_Relaxed_loopless_FVA_100kT_reran.csv')
# tr_250 = load_csv_and_cleanup('./flux_results/flux_sampling/flux_sample_TR_250_Relaxed_loopless_FVA_100kT.csv')
# tr_750 = load_csv_and_cleanup('./flux_results/flux_sampling/flux_sample_TR_750_Relaxed_loopless_FVA_100kT.csv')
# tr_1500 = load_csv_and_cleanup('./flux_results/flux_sampling/flux_sample_TR_1500_Relaxed_loopless_FVA_100kT.csv')


#Test the script on one CSV

wt_250 = mm.load_csv_and_cleanup('./flux_results/flux_sampling/8th_attempt_fixed_transporter_issues/10k_samples/WT_250.csv' ,tol=1e-4)
wt_750 = mm.load_csv_and_cleanup('./flux_results/flux_sampling/8th_attempt_fixed_transporter_issues/10k_samples/WT_750.csv' ,tol=1e-4)
wt_1500 = mm.load_csv_and_cleanup('./flux_results/flux_sampling/8th_attempt_fixed_transporter_issues/10k_samples/WT_1500.csv',tol=1e-4)
tr_250 = mm.load_csv_and_cleanup('./flux_results/flux_sampling/8th_attempt_fixed_transporter_issues/10k_samples/TR_250.csv',tol=1e-4)
tr_750 = mm.load_csv_and_cleanup('./flux_results/flux_sampling/8th_attempt_fixed_transporter_issues/10k_samples/TR_750.csv', tol=1e-4)
tr_1500 = mm.load_csv_and_cleanup('./flux_results/flux_sampling/8th_attempt_fixed_transporter_issues/10k_samples/TR_1500.csv',tol=1e-4)


    

df_list = [wt_250, wt_750, wt_1500, tr_250, tr_750, tr_1500]
names = ['wt_250', 'wt_750', 'wt_1500', 'tr_250', 'tr_750', 'tr_1500']
wt_list = [wt_250, wt_750, wt_1500]
tr_list = [tr_250, tr_750, tr_1500]
_250_list = [wt_250,tr_250]
_750_list = [wt_750,tr_750]
_1500_list=[wt_1500,tr_1500]


(10008, 697)
(10008, 814)
(10008, 750)
(10008, 1036)
(10008, 1119)
(10008, 1185)


First, print out all active reactions in both M and BS cells and their mean flux sums, just to show the rough disparity between the network fluxes between the two compartments

STAT NO. 2: Pairwise comparison of flux distributions
This offers a more comprehensive view on which reactions have a different flux distribution in between samples. The test used for this stat is the Kruskal-Wallis ranked test, which is a non-parametric test typically used for flux sampling to compare fluxes between groups


STAT NO 3: CORRELATION MATRICES
This code block is used to generate correlation matrices for the purpose of inferring which reactions are directly correlated with each other or not.

In [6]:
#Generation of Correlation Matrices 

wt_250_corr = wt_250.corr()
wt_750_corr = wt_750.corr()
wt_1500_corr = wt_1500.corr()

tr_250_corr = tr_250.corr()
tr_750_corr =  tr_750.corr()
tr_1500_corr = tr_1500.corr()

Generation of histograms and detailed analysis of Individual reactions at different light levels

These code blocks are for the analysis of the following reactions:
- Glycine Decarboxylase Activity
- Aminomethyltransferase Activity
- Correlation analysis for GLYDHDm for trans reactions (to identify which reactions are closely related to it then identify which contributes strongly to its reduction @ high levels)


STAT NO. 3: FLUX COUPLING ANALYSIS

This is a technique that is used to detect couplings between specific reactions in a given model. This technique is elaborated upon by Price, SChellenberger & Palsson (2004) in which a randomly sampled solution space, appropriately constrained based on environmental and phenotypic conditions, allows us to infer pairwise relations between the flux solutions. In turn, this method allows us to see which reactions are strongly coupled with each other given a set of constraints. 

Here are some reactions that I am particularly interested in, since I've found that they were unexpectedly decoupled from each other:
-PSIINC_M
-RBPCs_BS


These are interesting results:

- I found out that Cyclic Electron Transport flow is increased under high light conditions in the TR model 

In [5]:
filter_correlation_pairs(tr_250_corr, 0.2, ['MDHys_M'])

#If negative, higher MDHys malate synthesis. If positive, lower Malate synthesis.
#For positive correlations, higher flux == lower Malate synthesis. 

#Eaxample = Higher FNOR == Higher NADPH levels ==  higher MDH activity


NameError: name 'tr_250_corr' is not defined

In [5]:
#Generate correlation subplots for a number of reactions


def smallest_square_above(target):
    # Calculate the square root of the target number
    root = math.isqrt(target)
    
    # If the square of the root is less than or equal to the target, then increment the root by 1
    if root * root <= target:
        root += 1
    
    # Set the value of 'square' based on the value of 'root'
    if root == 2:
        square = 4
    else:
        square = root * root
    
    # Return the calculated square value
    return square


def generate_correlation_subplots(df, rxn, rxn_list_comparison, df_name='', size_mult=2, title='', directory='./plots/correlation_plots/'):
    # Remove duplicates then convert back to list
    rxn_list_comparison = list(set(rxn_comp for rxn_comp in rxn_list_comparison if rxn_comp in df.columns and rxn_comp != rxn))
    
    if len(rxn_list_comparison) == 0:
        print('no reactions to plot!')
        return None
    
    dims = int(np.sqrt(smallest_square_above(len(rxn_list_comparison))))
    
    # Initialize subplots
    fig, axes = plt.subplots(nrows=dims, ncols=dims, figsize=(size_mult*dims, size_mult*dims))
    
    # Set aspect ratio to be equal
    plt.gca().set_aspect('equal', adjustable='box')
    
    # Zip rxn_list_comparison and axes
    zip_values = itertools.zip_longest(axes.flat, rxn_list_comparison, fillvalue=None)
    
    for ax, rxn_compare in zip_values:
        if rxn_compare is not None:
            # Scatterplot of the given 'rxn' vs 'rxn_compare'
            g = sns.scatterplot(data=df, x=rxn, y=rxn_compare, ax=ax)
            
            # Fit a linear regression line
            slope, intercept, r_value, p_value, std_err = linregress(df[rxn], df[rxn_compare])
            
            # Add the regression line
            ax.plot(df[rxn], slope * df[rxn] + intercept, color='red')
            
            # Set labels and title
            g.set(xlabel=rxn, ylabel=rxn_compare)
            
            # Define bbox color based on absolute value of r_value
            if abs(r_value) > 0.6:
                bbox_color = 'red'
            elif abs(r_value) > 0.4:
                bbox_color = 'yellow'
            else:
                bbox_color = 'white'
            
            # Box for R value
            text_box = ax.text(0.95, 0.95, f'R = {r_value:.2f}', transform=ax.transAxes, ha='right', va='top', fontsize=11,
                               bbox=dict(facecolor=bbox_color, edgecolor='black', boxstyle='round,pad=0.2'))
            
        else:
            ax.margins(x=0, y=0)
            ax.set_axis_off()
            fig.delaxes(ax) 
    
    plt.tight_layout()
    plt.suptitle(f'{title}')
    plt.subplots_adjust(top=0.95, bottom=0.05, hspace=0.05)
    
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    
    filename = f'{title}.png'
    plot_path = os.path.join(directory, filename)
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f'Saved plot to {plot_path}')
    plt.close()


In [94]:
mets_of_interest = get_aggregate_reactions_mets(['glu-L_s0', 'asp-L_s0', 'nadph_s0', 'nadh_s0', 'oaa_s0', 'nh4_s0', 'co2_s0', 'mal-L_s0', 'oaa_c0', 'glu-L_c0'])


df_list = [tr_250, tr_750, tr_1500]
names = ['TR_250','TR_750','TR_1500']
#Generate correlation plots per items
for key, rxns_from_mets in mets_of_interest.items():
    metabolite_name = trans_model.metabolites.get_by_id(key).id
    
    
    for i in range(len(df_list)):
        df = df_list[i]
        df_name = names[i]
        generate_correlation_subplots(df, 'MDHys_M', rxns_from_mets, df_name=df_name,size_mult=2.5, title=f'Correlation analysis for {key} ({df_name})', directory=f'./plots/correlation_plots/{key}')
        


Saved plot to ./plots/correlation_plots/glu-L_c0/Correlation analysis for glu-L_c0 (TR_250).png
Saved plot to ./plots/correlation_plots/glu-L_c0/Correlation analysis for glu-L_c0 (TR_750).png
Saved plot to ./plots/correlation_plots/glu-L_c0/Correlation analysis for glu-L_c0 (TR_1500).png
Saved plot to ./plots/correlation_plots/glu-L_s0/Correlation analysis for glu-L_s0 (TR_250).png
Saved plot to ./plots/correlation_plots/glu-L_s0/Correlation analysis for glu-L_s0 (TR_750).png
Saved plot to ./plots/correlation_plots/glu-L_s0/Correlation analysis for glu-L_s0 (TR_1500).png
Saved plot to ./plots/correlation_plots/co2_s0/Correlation analysis for co2_s0 (TR_250).png
Saved plot to ./plots/correlation_plots/co2_s0/Correlation analysis for co2_s0 (TR_750).png
Saved plot to ./plots/correlation_plots/co2_s0/Correlation analysis for co2_s0 (TR_1500).png
Saved plot to ./plots/correlation_plots/nadh_s0/Correlation analysis for nadh_s0 (TR_250).png
Saved plot to ./plots/correlation_plots/nadh_s0/Cor

In [96]:
calvin_cycle_rxns = [rxns.id for rxns in trans_model.reactions if 'Calvin cycle' in rxns.notes['SUBSYSTEM']]


for i in range(len(df_list)):
    df = df_list[i]
    df_name = names[i]
 
        #Generate correlation plots for Calvin cycle reactions
    generate_correlation_subplots(df, 'MDHys_M', calvin_cycle_rxns, df_name=df_name, size_mult=2.5, title=f'Correlation analysis for MDHys_M and Calvin Cycle Reactions ({df_name})', directory=f'./plots/correlation_plots/Calvin_cycle')


    
photosynth_rxns = [rxns.id for rxns in trans_model.reactions if 'Photosynthesis' in rxns.notes['SUBSYSTEM']]
photorespiration_rxns = [rxns.id for rxns in trans_model.reactions if 'Glyoxylate' in rxns.notes['SUBSYSTEM'] or 'Photorespiration' in rxns.notes['SUBSYSTEM']]
nitrogen_rxns = [rxns.id for rxns in trans_model.reactions if 'Nitrogen' in rxns.notes['SUBSYSTEM']]


for i in range(len(df_list)):
    df = df_list[i]
    df_name = names[i]
 
        #Generate correlation plots for Calvin cycle reactions
    generate_correlation_subplots(df, 'MDHys_M', photosynth_rxns, df_name=df_name, size_mult=2.5, title=f'Correlation analysis for MDHys_M and Light reactions ({df_name})', directory=f'./plots/correlation_plots/Photosynthesis')
    generate_correlation_subplots(df, 'MDHys_M', photorespiration_rxns, df_name=df_name, size_mult=2.5, title=f'Correlation analysis for MDHys_M and Photorespiration and Glyoxylate Cycle({df_name})', directory=f'./plots/correlation_plots/Photorespiration')
    generate_correlation_subplots(df, 'RBPCs_BS', photosynth_rxns, df_name=df_name, size_mult=2.5, title=f'Correlation analysis for BS Rubisco and Light reactions ({df_name})', directory=f'./plots/correlation_plots/Photosynthesis')
    generate_correlation_subplots(df, 'RBPCs_BS', photorespiration_rxns, df_name=df_name, size_mult=2.5, title=f'Correlation analysis for BS Rubisco and Photorespiration and Glyoxylate Cycle({df_name})', directory=f'./plots/correlation_plots/Photorespiration')
    generate_correlation_subplots(df, 'MDHys_M', nitrogen_rxns, df_name=df_name, size_mult=2.5, title=f'Correlation analysis for MDHys_M and Nitrogen metabolism rxns ({df_name})', directory=f'./plots/correlation_plots/Nitrogen_mets')
    generate_correlation_subplots(df, 'RBPCs_BS', nitrogen_rxns, df_name=df_name, size_mult=2.5, title=f'Correlation analysis for BS Rubisco and Nitrogen metabolism rxns({df_name})', directory=f'./plots/correlation_plots/Nitrogen_mets')
 
    
#No reactions in Calvin Cycle seen to be correlated with MDHys

Saved plot to ./plots/correlation_plots/Calvin_cycle/Correlation analysis for MDHys_M and Calvin Cycle Reactions (TR_250).png
Saved plot to ./plots/correlation_plots/Calvin_cycle/Correlation analysis for MDHys_M and Calvin Cycle Reactions (TR_750).png
Saved plot to ./plots/correlation_plots/Calvin_cycle/Correlation analysis for MDHys_M and Calvin Cycle Reactions (TR_1500).png
Saved plot to ./plots/correlation_plots/Photosynthesis/Correlation analysis for MDHys_M and Light reactions (TR_250).png
Saved plot to ./plots/correlation_plots/Photorespiration/Correlation analysis for MDHys_M and Photorespiration and Glyoxylate Cycle(TR_250).png
Saved plot to ./plots/correlation_plots/Photosynthesis/Correlation analysis for BS Rubisco and Light reactions (TR_250).png
Saved plot to ./plots/correlation_plots/Photorespiration/Correlation analysis for BS Rubisco and Photorespiration and Glyoxylate Cycle(TR_250).png
Saved plot to ./plots/correlation_plots/Nitrogen_mets/Correlation analysis for MDHys_

In [104]:
all_malate_rxns = get_aggregate_reactions_mets(['mal-L'])
for key, val in all_malate_rxns.items():
    
    metabolite_name = trans_model.metabolites.get_by_id(key).id
    for i in range(len(df_list)):
        df = df_list[i]
        df_name = names[i]
        #Generate correlation plots for Calvin cycle reactions
    
        generate_correlation_subplots(df, 'mal-L_pd', val, df_name=df_name, size_mult=2.5, title=f'Correlation analysis for PD Malate transport and  Malate reactions {metabolite_name} ({df_name})', directory=f'./plots/correlation_plots/Malate_pd/{metabolite_name}')

 
    
#No reactions in Calvin Cycle seen to be correlated with MDHys

Saved plot to ./plots/correlation_plots/Malate_pd/mal-L_c0/Correlation analysis for PD Malate transport and  Malate reactions mal-L_c0 (TR_250).png
Saved plot to ./plots/correlation_plots/Malate_pd/mal-L_c0/Correlation analysis for PD Malate transport and  Malate reactions mal-L_c0 (TR_750).png
Saved plot to ./plots/correlation_plots/Malate_pd/mal-L_c0/Correlation analysis for PD Malate transport and  Malate reactions mal-L_c0 (TR_1500).png
Saved plot to ./plots/correlation_plots/Malate_pd/mal-L_s0/Correlation analysis for PD Malate transport and  Malate reactions mal-L_s0 (TR_250).png
Saved plot to ./plots/correlation_plots/Malate_pd/mal-L_s0/Correlation analysis for PD Malate transport and  Malate reactions mal-L_s0 (TR_750).png
Saved plot to ./plots/correlation_plots/Malate_pd/mal-L_s0/Correlation analysis for PD Malate transport and  Malate reactions mal-L_s0 (TR_1500).png
Saved plot to ./plots/correlation_plots/Malate_pd/mal-L_x0/Correlation analysis for PD Malate transport and  M

In [9]:
calvin_cycle_rxns = [rxns.id for rxns in trans_model.reactions if 'Calvin cycle' in rxns.notes['SUBSYSTEM']]
df_list = [tr_250, tr_750, tr_1500]
names = ['TR_250','TR_750','TR_1500']


for i in range(len(df_list)):
    df = df_list[i]
    df_name = names[i]


    # Generate correlation plots for Calvin cycle reactions
    generate_correlation_subplots(df, 'ATPSs_M', calvin_cycle_rxns, df_name=df_name, size_mult=2.5, 
                                  title=f'Correlation analysis for ATP Synthase and Calvin Cycle reactions ({df_name})',
                                  directory=f'./plots/correlation_plots/Calvin_Cycle')


Saved plot to ./plots/correlation_plots/Calvin_Cycle/Correlation analysis for ATP Synthase and Calvin Cycle reactions (TR_250).png
Saved plot to ./plots/correlation_plots/Calvin_Cycle/Correlation analysis for ATP Synthase and Calvin Cycle reactions (TR_750).png
Saved plot to ./plots/correlation_plots/Calvin_Cycle/Correlation analysis for ATP Synthase and Calvin Cycle reactions (TR_1500).png


In [13]:
photosynthesis_rxns = [rxns.id for rxns in trans_model.reactions if 'Photosynthesis' in rxns.notes['SUBSYSTEM']]

photorespiration_rxns = [rxns.id for rxns in trans_model.reactions if 'Glyoxylate' in rxns.notes['SUBSYSTEM'] or 'Photorespiration' in rxns.notes['SUBSYSTEM']]
nitrogen_rxns = [rxns.id for rxns in trans_model.reactions if 'Nitrogen' in rxns.notes['SUBSYSTEM']]


df_list = [tr_250, tr_750, tr_1500]
names = ['TR_250','TR_750','TR_1500']
#Generate correlation plots per items
    

for i in range(len(df_list)):
    df = df_list[i]
    df_name = names[i]
    generate_correlation_subplots(df, 'MDHys_BS', photosynthesis_rxns, df_name=df_name,size_mult=2.5, title=f'Correlation analysis for MDHys_BS and Light reactions ({df_name})', directory=f'./plots/correlation_plots/BS_tissue')

    generate_correlation_subplots(df, 'MDHys_BS', calvin_cycle_rxns, df_name=df_name,size_mult=2.5, title=f'Correlation analysis for MDHys_BS and Calvin Cycle ({df_name})', directory=f'./plots/correlation_plots/BS_tissue')

    generate_correlation_subplots(df, 'MDHys_BS', photorespiration_rxns, df_name=df_name,size_mult=2.5, title=f'Correlation analysis for MDHys_BS and Photorespiration ({df_name})', directory=f'./plots/correlation_plots/BS_tissue')

    generate_correlation_subplots(df, 'MDHys_BS', nitrogen_rxns, df_name=df_name,size_mult=2.5, title=f'Correlation analysis for MDHys_BS and Nitrogen Rxns ({df_name})', directory=f'./plots/correlation_plots/BS_tissue')


Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis for MDHys_BS and Light reactions (TR_250).png
Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis for MDHys_BS and Calvin Cycle (TR_250).png
Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis for MDHys_BS and Photorespiration (TR_250).png
Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis for MDHys_BS and Nitrogen Rxns (TR_250).png
Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis for MDHys_BS and Light reactions (TR_750).png
Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis for MDHys_BS and Calvin Cycle (TR_750).png
Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis for MDHys_BS and Photorespiration (TR_750).png
Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis for MDHys_BS and Nitrogen Rxns (TR_750).png
Saved plot to ./plots/correlation_plots/BS_tissue/Correlation analysis f

Hmm. PSII is more strongly coupled with reactions involving Glycolysis/Gluconeogenesis in the TR1500 than in WT1500. I wonder if there are any depressed fluxes dun sa group na yun?

Correlation of flux measurements: use in designing experiments
Uniform random sampling of the steady-state flux space allows for the calculation of the correlation coefficient (rij) between any two fluxes (vi and vj) in the network. Thus, sampling provides a straightforward means of not only calculating perfectly correlated subsets (
), but also of identifying well-correlated, but not perfectly correlated reaction sets. The matrix of squared pairwise correlation coefficients for all the RBC metabolic fluxes was computed. The fluxes can be ordered such that the “correlated reaction sets” (defined here as 
) are listed in order of decreasing number of fluxes in each set (Table 1).


From Price, Schellenberger and Palsson, (2004) -- Uniform Sammpling of flux

In [102]:
trans_model.reactions.DICtm_M

Reaction identifier,DICtm_M
Name,"dicarboxylate transport, mitochondrial"
Memory address,0x7fa743e7a650
Stoichiometry,mal-L_m0 + succ_c0 --> mal-L_c0 + succ_m0 L-Malate + Succinate --> L-Malate + Succinate
GPR,LOC_Os01g14520 or LOC_Os01g28840 or LOC_Os01g43460 or LOC_Os01g12680 or LOC_Os04g48530 or...
Lower bound,0.0
Upper bound,1000000.0






Hmm. Interesting results. It shows that fluxes have become more decoupled in the low light regimens compared with medium and high light regimens when comparing results between parametrizations (WT and TR), at a threshold value of 0.001.
